### Spelling Corrector

In [1]:
import re
from collections import Counter
from pprint import pprint

In [2]:
text = open('big.txt').read()

In [3]:
def words(text):
    return re.findall(r'\w+', text.lower()) # '\w': get characters and numbers (splited digit by digit) in the text
                                            # '\w+': get only words and numbers (splited by space) in the text
    
word_count = Counter(words(text))
N = sum(word_count.values())

print('There are %d words in the text.' % N)
word_count.most_common(10)

There are 1115585 words in the text.


[('the', 79809),
 ('of', 40024),
 ('and', 38312),
 ('to', 28765),
 ('in', 22023),
 ('a', 21124),
 ('that', 12512),
 ('he', 12401),
 ('was', 11410),
 ('it', 10681)]

In [4]:
def P(word):
    # Probability of 'word'.
    return word_count[word] / N

print(list(map(lambda x: (x, P(x)), words('speling spelling speeling'))))

[('speling', 0.0), ('spelling', 3.585562731660967e-06), ('speeling', 0.0)]


In [5]:
class spelling_corrector(object):
    
    def __init__(self):
        self.letters = bytearray(range(97,123)).decode("utf-8")
        self.WORDS = Counter(words(text))
    
    def edits1(self, word):
        # All edits that are one edit away from 'word'.
        splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
        deletes    = [L + R[1:]               for L, R in splits if R]
        transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
        replaces   = [L + c + R[1:]           for L, R in splits if R for c in self.letters]
        inserts    = [L + c + R               for L, R in splits for c in self.letters]
        return set(deletes + transposes + replaces + inserts)

    def edits2(self, word): 
        # All edits that are two edits away from 'word'.
        return (e2 for e1 in self.edits1(word) for e2 in self.edits1(e1))

    def known(self, words): 
        # The subset of 'words' that appear in the dictionary of WORDS.
        return set(w for w in words if w in self.WORDS)

    def candidates(self, word): 
        # Generate possible spelling corrections for word.
        return (self.known([word]) or self.known(self.edits1(word)) or self.known(self.edits2(word)) or [word])

    def correction(self, word): 
        # Most probable spelling correction for word.
        return max(self.candidates(word), key=P)

In [6]:
sc = spelling_corrector()

print('speling -->', sc.correction('speling'))
print('korrectud -->', sc.correction('korrectud'))

speling --> spelling
korrectud --> corrected


### Practice : deal with fusion errors and multi-token errors

In [7]:
class spelling_corrector(object):
    
    def __init__(self):
        self.letters = bytearray(range(97,123)).decode("utf-8")
        self.WORDS = Counter(words(text))
    
    def edits1(self, word):
        # All edits that are one edit away from 'word'.
        splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]                
        deletes    = [L + R[1:]               for L, R in splits if R]
        transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
        replaces   = [L + c + R[1:]           for L, R in splits if R for c in self.letters]
        inserts    = [L + c + R               for L, R in splits for c in self.letters]
        return set(deletes + transposes + replaces + inserts)

    def edits2(self, word): 
        # All edits that are two edits away from 'word'.
        return (e2 for e1 in self.edits1(word) for e2 in self.edits1(e1))

    def known(self, words): 
        # The subset of 'words' that appear in the dictionary of WORDS.
        return set(w for w in words if w in self.WORDS)

    def candidates(self, word): 
        # Generate possible spelling corrections for word.
        return (self.known([word]) or self.known(self.edits1(word)) or self.known(self.edits2(word)) or [word])

    def correction(self, word): 
        # Most probable spelling correction for word.
        word = "".join(word.split())
        
        if word in self.WORDS:
            return word

        splits = [(word[:i], word[i:]) for i in range(len(word) + 1)]
        
        correction = []
        for i in range(len(splits)):
            if splits[i][0] in self.WORDS and splits[i][1] in self.WORDS:
                correction.append(splits[i][0])
                correction.append(splits[i][1])
        correction = ' '.join(correction)
        
        if len(correction) != 0:
            return correction
        
        return max(self.candidates(word), key=P)

In [8]:
sc = spelling_corrector()

print('speling -->', sc.correction('speling'))
print('korrectud -->', sc.correction('korrectud'))
print('taketo -->', sc.correction('taketo'))
print('mor efun -->', sc.correction('mor efun'))
print('with out -->', sc.correction('with out'))

speling --> spelling
korrectud --> corrected
taketo --> take to
mor efun --> more fun
with out --> without
